<a href="https://colab.research.google.com/github/RodolfoFerro/cimps20/blob/main/notebooks/04-Redes_neuronales_convolucionales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes neuronales convolucionales

> **Rodolfo Ferro** <br>
> Google Dev Expert en ML, 2020.
>
> _Redes:_
> - GitHub - [RodolfoFerro](https://github.com/RodolfoFerro)
> - Twitter - [@FerroRodolfo](https://twitter.com/FerroRodolfo)
> - Instagram - [@rodo_ferro](https://instagram.com/rodo_ferro)

## Contenidos

#### **Sección V**
1. **Código:** El dataset de modas
2. **Código:** Preparación de datos
3. **Código:** Creación del modelo
4. **Código:** Entrenamiento del modelo
5. **Código:** Evaluación del modelo
6. **Código:** Predicción


## **Sección V**

### El dataset de modas

Comencemos importando TensorFlow.

In [ ]:
import tensorflow as tf
print(tf.__version__)

Los datos de Fashion MNIST están disponibles directamente en la API de conjuntos de datos de `tf.keras`. Los cargas así:

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

Llamar a `load_data` en este objeto nos dará dos conjuntos con los valores de entrenamiento y prueba para los gráficos que contienen las prendas y sus etiquetas.

In [ ]:
(training_images, training_labels), (test_images, test_labels) = fashion_mnist.load_data()

¿Cómo se ven estos valores?

Imprimamos una imagen de entrenamiento y una etiqueta de entrenamiento para ver.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(linewidth=200)


# Set index of image to be seen
img_index = 0

# Plot image
plt.imshow(training_images[img_index], cmap='gray')
plt.axis(False)

print("Label:", training_labels[img_index])
print("Matrix:", training_images[img_index])

### Preparación de los datos

Notarás que todos los valores están entre 0 y 255. Si estamos entrenando una red neuronal, por varias razones es más fácil si transformamos los valores para tratar todos con valores entre 0 y 1. Este proceso se llama **normalización**.

Además, para este proceso añadiremos la expansión de dimensiones para poder alimentar a la red.

In [ ]:
training_images = training_images.reshape(60000, 28, 28, 1)
training_images = training_images / 255.0

test_images = test_images.reshape(10000, 28, 28, 1)
test_images = test_images/255.0

### Creación del modelo

En el cuaderno anterior vimos cómo hacer el reconocimiento de moda usando una red neuronal profunda (DNN) que contiene tres capas: la capa de entrada (en la forma de los datos), la capa de salida (en la forma de la salida deseada) y una capa oculta. 

Idealmente experimentamos con el impacto de diferentes tamaños de capa oculta, número de épocas de entrenamiento, etc., para ver estos resultados reflejados en la precisión final.

In [ ]:
model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(), 
            tf.keras.layers.Dense(128, activation=tf.nn.relu), 
            tf.keras.layers.Dense(10, activation=tf.nn.softmax)
        ])

Tu precisión es probablemente de aproximadamente el 89% en el entrenamiento y el 87% en la validación, que no está mal... ¿Pero cómo podemos mejorarlo? 

Una forma es usar capas convolucionales en la red.**Estas operaciones reducen el contenido de la imagen para centrarse en detalles específicos y distintos.**

Si alguna vez has procesado imágenes con un filtro (como este: https://en.wikipedia.org/wiki/Kernel_(image_processing)), las convoluciones te resultarán muy familiares.

En resumen, pasaremos una matriz (generalmente 3x3 o 5x5) sobre la imagen. Al cambiar los píxeles subyacentes según la fórmula dentro de esa matriz, podemos hacer cosas como la detección de bordes.

Esto es perfecto para la visión por computadora, porque a menudo son las características que pueden resaltarse de esta manera las que distinguen un elemento de otro, y la cantidad de información necesaria es mucho menor porque sólo entrenaremos la red sobre las características resaltadas.

Ese es el concepto de redes neuronales convolucionales. Agreguemos algunas capas para hacer la convolución antes de tener las capas densas, así la información que va a las capas densas está más enfocada y posiblemente más precisa.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

### Entrenamiento del modelo

In [ ]:
model.compile(
    optimizer=tf.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(training_images, training_labels, epochs=5)

### Evaluación del modelo

In [ ]:
model.evaluate(test_images, test_labels)

### Predicción


In [ ]:
test_index = 30

plt.imshow(np.squeeze(test_images[test_index]), cmap='gray')
plt.axis(False)

print("Label:", test_labels[test_index])
prediction = model.predict(np.expand_dims(test_images[test_index], axis=0))
print("Prediction:", np.argmax(prediction))